#### Defined by User

In [ ]:
## Define the model monitor name here that you will be creating 
MONITOR_NAME = "mm-demo"
## Specify the data source [aws_s3,sql,local]
DATA_SOURCE = "local"
## Specify Training type, [training,retraining]
INPUT_TRAIN_TYPE = "retraining"
## Set CLEANUP = True, after your experiment is complete.
CLEANUP = False

info = [MONITOR_NAME,DATA_SOURCE,INPUT_TRAIN_TYPE]

#### Dkube Resources

In [ ]:
import os, time,json,shutil
from dkube.sdk import *

In [ ]:
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
username =  os.getenv("USERNAME")
dkube_url = os.getenv("DKUBE_URL")
api = DkubeApi(URL=dkube_url,token=token)

#### Code

In [ ]:
try:
    code = DkubeCode(username, name="insurance")
    code.update_git_details("https://github.com/oneconvergence/dkube-examples.git",branch="monitoring")
    api.create_code(code)

except Exception as e:
    if e.reason.lower()!="conflict":
        response = e.body
        print(f"Failed[{response.code}]: {response.message}")

#### Dataset

In [ ]:
if DATA_SOURCE == 'local':
    try:
        dataset = DkubeDataset(username, name='insurance-data')
        dataset.update_git_details(url="https://dkube-examples-data.s3.us-west-2.amazonaws.com/monitoring-insurance/training-data/insurance.csv")
        dataset.update_dataset_source(source="pub_url")
        api.create_dataset(dataset)

    except Exception as e:
        if e.reason.lower()!="conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")

In [ ]:
if DATA_SOURCE == 'aws_s3':
    try:
        dataset = DkubeDataset(user, name='insurance-data',remote=True)
        dataset.update_dataset_source('aws_s3')
        dataset.update_awss3_details(bucket="mm-workflow",prefix=mm_name,key=access_key,secret=secret_key)
        api.create_dataset(dataset)

    except Exception as e:
        if e.reason.lower()!="conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")       

#### Model

In [ ]:
try:
    model = DkubeModel(username, name='insurance-model')
    model.update_model_source(source='dvs')
    api.create_model(model)
    
except Exception as e:
    if e.reason.lower()!="conflict":
        response = e.body
        print(f"Failed[{response.code}]: {response.message}")

#### Saving MM-Details

In [ ]:
HOME = '/home/'+username+'/workspace/insurance/insurance'

In [ ]:
with open("details.txt","w") as f:
    json.dump(info,f)

In [ ]:
shutil.copy('details.txt',HOME)

#### Cleanup

In [ ]:
if CLEANUP:
    api.delete_code(username,'insurance',force=True)
    api.delete_dataset(username,'insurance-data',force=True)
    api.delete_model(usename,'insurance-model',force=True)